In [3]:
import pandas as pd
df = pd.read_csv('test.csv',encoding='unicode_escape')

In [4]:
df

,Questions,Answers
0,What is data analytics?,Data analytics is the process of examining raw...
1,What are the main methods used in data analytics?,Data analytics encompasses various methods suc...
2,How is data analytics used in business?,Data analytics is widely used in business to g...
3,What are the key steps in the data analytics p...,The data analytics process typically involves ...
4,What is the role of machine learning in data a...,Machine learning plays a crucial role in data ...
5,What is the importance of data quality in data...,Data quality is vital in data analytics as the...
6,What are the ethical considerations in data an...,Ethical considerations in data analytics inclu...
7,What are some common challenges in data analyt...,Common challenges in data analytics include de...


In [5]:
question_list = df['Questions']

In [6]:
question_list

0                              What is data analytics?
1    What are the main methods used in data analytics?
2              How is data analytics used in business?
3    What are the key steps in the data analytics p...
4    What is the role of machine learning in data a...
5    What is the importance of data quality in data...
6    What are the ethical considerations in data an...
7    What are some common challenges in data analyt...
Name: Questions, dtype: object

In [7]:
question_list = df['Questions'].tolist()

In [8]:
question_list

['What is data analytics?',
 'What are the main methods used in data analytics?',
 'How is data analytics used in business?',
 'What are the key steps in the data analytics process?',
 'What is the role of machine learning in data analytics?',
 'What is the importance of data quality in data analytics?',
 'What are the ethical considerations in data analytics?',
 'What are some common challenges in data analytics?']

In [9]:
answer_list = df['Answers']

In [10]:
answer_list

0    Data analytics is the process of examining raw...
1    Data analytics encompasses various methods suc...
2    Data analytics is widely used in business to g...
3    The data analytics process typically involves ...
4    Machine learning plays a crucial role in data ...
5    Data quality is vital in data analytics as the...
6    Ethical considerations in data analytics inclu...
7    Common challenges in data analytics include de...
Name: Answers, dtype: object

In [11]:
answer_list = df['Answers'].tolist()

In [12]:
answer_list

['Data analytics is the process of examining raw data to discover meaningful patterns, draw conclusions, and make informed decisions. It involves collecting, transforming, and analyzing large sets of data to extract valuable insights and support decision-making.',
 'Data analytics encompasses various methods such as descriptive analytics, which focuses on summarizing and interpreting historical data; predictive analytics, which uses statistical models and machine learning algorithms to make future predictions; and prescriptive analytics, which provides recommendations and strategies based on the analysis of data.',
 'Data analytics is widely used in business to gain insights into customer behavior, optimize operations, improve marketing strategies, identify trends, forecast demand, and make data-driven decisions. It helps businesses enhance efficiency, identify opportunities, and gain a competitive edge.',
 'The data analytics process typically involves several steps: data collection, 

In [13]:
import nltk
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [46]:
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

def preprocess(text):
  lemmatizer = WordNetLemmatizer()
  stemmer = PorterStemmer()
  text = re.sub(r'[^\w\s]','',text)
  tokens = nltk.word_tokenize(text.lower())
  tokens = [tokens for tokens in tokens if tokens not in stopwords.words('english')]
  lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
  stemmed_tokens = [stemmer.stem(token) for token in lemmatized_tokens]
  return ' '.join(stemmed_tokens)

In [47]:
def preprocess_with_stopwords(text):
  lemmatizer = WordNetLemmatizer()
  stemmer = PorterStemmer()
  text = re.sub(r'[^\w\s]','',text)
  tokens = nltk.word_tokenize(text.lower())
  lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
  stemmed_tokens = [stemmer.stem(token) for token in lemmatized_tokens]
  return ' '.join(stemmed_tokens)

In [48]:
vectorizer = TfidfVectorizer(tokenizer=nltk.word_tokenize)
x = vectorizer.fit_transform([preprocess(q) for q in question_list])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [35]:
vectorizer = TfidfVectorizer(tokenizer=nltk.word_tokenize)
x = vectorizer.fit_transform([preprocess(q) for q in question_list])

def get_response(text):
  processed_text = preprocess_with_stopwords(text)
  print("processed_text:",processed_text)
  vectorized_text = vectorizer.transform([processed_text])
  similarities = cosine_similarity(vectorized_text,x)
  print("similarities:",similarities)
  max_similarity = np.max(similarities)
  print("max_similarity:",max_similarity)
  if max_similarity > 0.6:
    high_similarity_questions = [q for q , s in zip(question_list,similarities[0]) if s>0.6]
    print("high_similarity_questions:",high_similarity_questions)

    target_answer = []
    for q in high_similarity_questions:
        q_index = question_list.index(q)
        target_answer.append(answer_list[q_index])
    print(target_answer)

    z = vectorizer.fit_transform([preprocess_with_stopwords(q) for q in high_similarity_questions])
    processed_text_with_stopwords = preprocess_with_stopwords(text)
    print('processed_text_with_stopwords:',processed_text_with_stopwords)
    vectorize_text_with_stopwords = vectorizer.transform([processed_text_with_stopwords])
    final_similarities = cosine_similarity(processed_text_with_stopwords,z)
    closest = np.argmax(final_similarities)
    return target_answer[closest]
  else:
    return "I can't answer this question."

get_response('who doni')



processed_text: who doni
similarities: [[0. 0. 0. 0. 0. 0. 0. 0.]]
max_similarity: 0.0


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


"I can't answer this question."

In [32]:
vectorizer = TfidfVectorizer(tokenizer=nltk.word_tokenize)
x = vectorizer.fit_transform([preprocess(q) for q in question_list])

def get_response(text):
  processed_text = preprocess_with_stopwords(text)
  print("processed_text:",processed_text)
  vectorized_text = vectorizer.transform([processed_text])
  similarities = cosine_similarity(vectorized_text,x)
  print("similarities:",similarities)
  max_similarity = np.max(similarities)
  print("max_similarity:",max_similarity)
  if max_similarity > 0.6:
    high_similarity_questions = [q for q , s in zip(question_list,similarities[0]) if s>0.6]
    print("high_similarity_questions:",high_similarity_questions)

    target_answer = []
    for q in high_similarity_questions:
        q_index = question_list.index(q)
        target_answer.append(answer_list[q_index])
    print(target_answer)

    z = vectorizer.fit_transform([preprocess_with_stopwords(q) for q in high_similarity_questions])
    processed_text_with_stopwords = preprocess_with_stopwords(text)
    print('processed_text_with_stopwords:',processed_text_with_stopwords)
    vectorize_text_with_stopwords = vectorizer.transform([processed_text_with_stopwords])
    final_similarities = cosine_similarity(processed_text_with_stopwords,z)
    closest = np.argmax(final_similarities)
    return target_answer[closest]
  else:
    return "I can't answer this question."

get_response('what is data analytics')



processed_text: what is data analyt
similarities: [[1.         0.32491362 0.39723569 0.3100033  0.3100033  0.50650236
  0.37086918 0.37086918]]
max_similarity: 1.0000000000000002
high_similarity_questions: ['What is data analytics?']
['Data analytics is the process of examining raw data to discover meaningful patterns, draw conclusions, and make informed decisions. It involves collecting, transforming, and analyzing large sets of data to extract valuable insights and support decision-making.']
processed_text_with_stopwords: what is data analyt


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


ValueError: could not convert string to float: 'what is data analyt'